In [13]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import cv2


In [2]:
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [3]:
train_dataset = datasets.CIFAR10(root = 'data', train = True, transform=transform, download= True)

Files already downloaded and verified


In [4]:
test_dataset = datasets.CIFAR10(root = 'data', train = False, transform = transform, download = True)

Files already downloaded and verified


In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= 64, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 64, shuffle = True)

In [6]:
for batch in train_loader:
    images, labels = batch
    print(images.size())
    break

torch.Size([64, 3, 32, 32])


In [7]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.cnn1 = nn.Conv2d(3,9,3,1)
        self.cnn2 = nn.Conv2d(9,27,3,1)
        self.cnn3 = nn.Conv2d(27,54,3,1)
        self.mp = nn.MaxPool2d(2,2)
        self.relu = nn.ReLU()
        self.fc = nn.Sequential(
            nn.Linear(54*4*4,4),
            nn.ReLU(),
            nn.Linear(4,16),
            nn.ReLU(),
            nn.Linear(16,10),
        )

    def forward(self, x):
        x = self.cnn1(x)
        x = self.relu(x)
        x = self.mp(x)
        x = self.cnn2(x)
        x = self.relu(x)
        x = self.mp(x)
        x = self.cnn3(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
        


In [8]:

model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay = 0.005, momentum = 0.9)  

epochs = 100
for epoch in range(epochs):
    #epoch_losses = []
    for i, data in enumerate(train_loader):

        inputs, labels = data

        outputs = model(inputs)
        loss = criterion(outputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #epoch_losses.append(loss.item())
    
    
    # Print average loss for the entire epoch
    # avg_loss = sum(epoch_losses) / len(epoch_losses)
    # print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.4f}')

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))



Epoch [1/100], Loss: 2.3270
Epoch [2/100], Loss: 2.2459
Epoch [3/100], Loss: 2.3541
Epoch [4/100], Loss: 2.0167
Epoch [5/100], Loss: 1.8683
Epoch [6/100], Loss: 1.9947
Epoch [7/100], Loss: 1.8389
Epoch [8/100], Loss: 1.9198
Epoch [9/100], Loss: 1.5633
Epoch [10/100], Loss: 1.8190
Epoch [11/100], Loss: 1.9674
Epoch [12/100], Loss: 1.8804
Epoch [13/100], Loss: 1.6202
Epoch [14/100], Loss: 1.2895
Epoch [15/100], Loss: 1.8376
Epoch [16/100], Loss: 1.5022
Epoch [17/100], Loss: 1.6492
Epoch [18/100], Loss: 1.7067
Epoch [19/100], Loss: 1.2387
Epoch [20/100], Loss: 2.0119
Epoch [21/100], Loss: 1.2682
Epoch [22/100], Loss: 1.7193
Epoch [23/100], Loss: 1.0650
Epoch [24/100], Loss: 1.2132
Epoch [25/100], Loss: 1.5972
Epoch [26/100], Loss: 1.4063
Epoch [27/100], Loss: 1.0898
Epoch [28/100], Loss: 1.0410
Epoch [29/100], Loss: 1.4882
Epoch [30/100], Loss: 1.5175
Epoch [31/100], Loss: 1.0144
Epoch [32/100], Loss: 1.4430
Epoch [33/100], Loss: 1.4326
Epoch [34/100], Loss: 1.1376
Epoch [35/100], Loss: 1

In [9]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the train images: {} %'.format(100 * correct / total))




Accuracy of the network on the train images: 69.436 %
